<a href="https://colab.research.google.com/github/buseskorkmaz/Sentiment-Analysis-with-Deep-Learning/blob/main/flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flair
!pip install allennlp==0.9.0

     |████████████████████████████████| 7.6MB 2.7MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 163kB 35.5MB/s 
     |████████████████████████████████| 133kB 47.0MB/s 
     |████████████████████████████████| 30.9MB 100kB/s 
     |████████████████████████████████| 5.3MB 15.8MB/s 
     |████████████████████████████████| 245kB 40.5MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 3.2MB 35.5MB/s 
     |████████████████████████████████| 2.1MB 33.5MB/s 
     |████████████████████████████████| 92kB 9.2MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 235kB 45.7MB/s 
  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5588 sha256=e018a8b952f4997e3a19c79cf7c7766b8cd2c814f02d5578fce5c12ce89f088f
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
  Created wheel 

In [ ]:
import pandas as pd
import tqdm
import numpy as np

In [ ]:
df = pd.read_json('financial_news_scored.json', orient = 'records', encoding='utf-8').sample(frac=1)
df['text'] = df['text'].map(lambda x: x.lstrip('BRIEF-'))
df.set_index('index',inplace=True)
# Optional lowercase for test data (if model was trained on lowercased text
df['text'] = df['text'].str.lower()
df['label'] = '__label__' + df['score'].astype(str)
df= df.drop(columns=['versionCreated', 'storyId', 'sourceCode', 'storyText','score'])
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

df.iloc[0:int(len(df)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
df.iloc[int(len(df)*0.8):int(len(df)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
df.iloc[int(len(df)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False)

df

,label,text
index,,
1588242145000,__label__1,va expects any potential future provisions aga...
1591886915578,__label__-1,itch revises anadolubank's outlook to negative...
1591286414000,__label__-1,akfen reit q1 net loss widens to 60.6 million ...
1591451123000,__label__-1,south africa's mtn faces new allegations in u....
1591770119000,__label__-1,global yatirim holding q1 net loss increases t...
...,...,...
1598690940000,__label__1,update 5-uae scraps israel boycott in new step...
1596037421000,__label__-1,marti reit q2 net result swings to loss of 11....
1598886714613,__label__-1,itch maintains negative rating watch on global...


In [ ]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from flair.embeddings import TransformerDocumentEmbeddings,TransformerWordEmbeddings
from flair.embeddings import BertEmbeddings, ELMoEmbeddings

# this is the folder in which train, test and dev files reside
data_folder = '/content/'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ClassificationCorpus(data_folder,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')
# print the number of Sentences in the train split
print(len(corpus.train))

# print the number of Sentences in the test split
print(len(corpus.test))

# print the number of Sentences in the dev split
print(len(corpus.dev))


2020-09-13 15:01:25,612 Reading data from /content
2020-09-13 15:01:25,614 Train: /content/train.txt
2020-09-13 15:01:25,620 Dev: /content/dev.txt
2020-09-13 15:01:25,622 Test: /content/test.txt
586
73
74


In [ ]:
from flair.embeddings import StackedEmbeddings

# init BERT base (cases)
#optional_embedding = BertEmbeddings('bert-base-uncased')
# OR init ELMo (original)
#optional_embedding = ELMoEmbeddings('original')

#word_embeddings = [
#    optional_embedding,
#    FlairEmbeddings('news-forward'),
#    FlairEmbeddings('news-backward')]


#word_embeddings = [WordEmbeddings('glove')]

#document_embeddings = DocumentRNNEmbeddings(
#        word_embeddings,
#        hidden_size=512,
#        reproject_words=True,
#        reproject_words_dimension=256
#    )

document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased',fine_tune=True)



In [ ]:
classifier = TextClassifier(document_embeddings, 
                            label_dictionary=corpus.make_label_dictionary(), 
                            multi_label=True)

trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=5,mini_batch_size=32)

2020-09-13 15:37:41,675 Computing label dictionary. Progress:


100%|██████████| 659/659 [00:00<00:00, 801.78it/s] 

2020-09-13 15:37:43,252 [b'0', b'-1', b'1']
2020-09-13 15:37:43,269 ----------------------------------------------------------------------------------------------------
2020-09-13 15:37:43,271 Model: "TextClassifier(
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, b

2020-09-13 15:37:44,804 epoch 1 - iter 1/19 - loss 0.69793844 - samples/sec: 38.94 - lr: 0.100000
2020-09-13 15:37:45,535 epoch 1 - iter 2/19 - loss 0.36785468 - samples/sec: 44.65 - lr: 0.100000
2020-09-13 15:37:46,292 epoch 1 - iter 3/19 - loss 0.24853520 - samples/sec: 42.50 - lr: 0.100000
2020-09-13 15:37:46,981 epoch 1 - iter 4/19 - loss 0.19249319 - samples/sec: 47.05 - lr: 0.100000
2020-09-13 15:37:47,690 epoch 1 - iter 5/19 - loss 0.15664401 - samples/sec: 45.24 - lr: 0.100000
2020-09-13 15:37:48,430 epoch 1 - iter 6/19 - loss 0.13338151 - samples/sec: 43.32 - lr: 0.100000
2020-09-13 15:37:49,257 epoch 1 - iter 7/19 - loss 0.11637191 - samples/sec: 41.02 - lr: 0.100000
2020-09-13 15:37:49,905 epoch 1 - iter 8/19 - loss 0.10350685 - samples/sec: 49.51 - lr: 0.100000
2020-09-13 15:37:50,572 epoch 1 - iter 9/19 - loss 0.09351362 - samples/sec: 48.05 - lr: 0.100000
2020-09-13 15:37:51,296 epoch 1 - iter 10/19 - loss 0.08490687 - samples/sec: 44.35 - lr: 0.100000
2020-09-13 15:37:51

{'dev_loss_history': [0.829311728477478,
  0.8733912706375122,
  0.9419206380844116,
  0.7649868726730347,
  0.9187461733818054],
 'dev_score_history': [0.6575, 0.6986, 0.7123, 0.6667, 0.7027],
 'test_score': 0.7534,
 'train_loss_history': [0.051038811448961496,
  0.006611294591015107,
  0.0029970998324355797,
  0.020630335741627374,
  0.011562082830718473]}